# Retrieving data on books' service 

## Goals of the Study
The goal of this study is to supply an information that can be used to generate a value proposition for a new product in the book market.
This will be done based on data of one of the services competing in this market, consisting of information about books, publishers, authors, and customer ratings and reviews of books.

## Initialization

In [1]:
import pandas as pd
! pip install psycopg2
from sqlalchemy import create_engine

db_config = {'user': 'user',         
             'pwd': 'password', 
             'host': 'host',
             'port': 1234,             
             'db': 'name of the database'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Studying the tables

In [9]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print("The {} table:".format(table))
    query = """
    SELECT *
    FROM {}
    LIMIT 10
    """.format(table)
    display(pd.read_sql_query(query, con = engine))

The books table:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


The authors table:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


The publishers table:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


The ratings table:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


The reviews table:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


## Completing the tasks

**1. Find the number of books released after January 1, 2000**

In [109]:
query = """
SELECT COUNT(book_id) as number_books
FROM books
WHERE publication_date > '2000-01-01';
"""
table1 =pd.io.sql.read_sql(query, con = engine)
f"The number of books released after January 1, 2000 is {table1.loc[0,'number_books']}."

'The number of books released after January 1, 2000 is 819.'

**2. Find the number of user reviews and the average rating for each book**

In [85]:
query = """
SELECT title as book_name, COUNT(DISTINCT rev.review_id) as number_reviews, AVG(rat.rating) as avg_rating
FROM books as b
LEFT JOIN reviews as rev ON rev.book_id=b.book_id
LEFT JOIN ratings as rat ON rat.book_id=rev.book_id
GROUP BY b.book_id
ORDER BY number_reviews DESC;
"""
pd.io.sql.read_sql(query, con = engine)

,book_name,number_reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,NaN
996,The Natural Way to Draw,0,NaN
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,NaN
998,Essential Tales and Poems,0,NaN


**3. Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).**

In [110]:
query = """
SELECT p.publisher
FROM publishers as p
RIGHT JOIN books as b ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 1;

"""
table2= pd.io.sql.read_sql(query, con = engine)
f"The publisher that has released the greatest number of books with more than 50 pages is {table2.loc[0,'publisher']}."

'The publisher that has released the greatest number of books with more than 50 pages Penguin Books.'

**4. Identify the author with the highest average book rating (look only at books with at least 50 ratings).**

In [111]:
query = """
SELECT author
FROM authors as a
RIGHT JOIN books as b ON b.author_id=a.author_id
RIGHT JOIN ratings as rat ON rat.book_id=b.book_id
GROUP BY a.author, rat.book_id
HAVING COUNT(rat.rating_id) >= 50
ORDER BY AVG(rat.rating) DESC
LIMIT 1;
"""
table3 = pd.io.sql.read_sql(query, con = engine)
f"The author with the highest average book rating (for books with at least 50 ratings) is {table3.loc[0,'author']}."

'The author with the highest average book rating (for books with at least 50 ratings) is J.K. Rowling/Mary GrandPré.'

**5. Find the average number of text reviews among users who rated more than 50 books.**

In [112]:
query = """
WITH rat as
(SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(book_id)>50),
rev as
(SELECT username, COUNT(text) as number_text
FROM reviews
GROUP BY username)

SELECT AVG(rev.number_text) as avg_number_text
FROM rat JOIN rev ON rat.username=rev.username;
"""
table4= pd.io.sql.read_sql(query, con = engine)
f"The average number of text reviews among users who rated more than 50 books is {table4.loc[0,'avg_number_text']}."

'The average number of text reviews among users who rated more than 50 books is 24.333333333333332.'